# Lab n: Classification

In [1]:
## Notebook Settings
# Add autotime of each block
!pip install ipython-autotime
%load_ext autotime

### Goals:
- Learn the basics of cyber network data with respect to consumer IoT devices
- Load network data into Pandas and a GDF (for comparison)
- Explore network data and features
- Use XGBoost to build a classification model
- Test the model

This lab builds on the previous labs and will utilize some of those skills.

### Background

#### The Internet of Things and Data at a Massive Scale

Gartner estimates there are currently over 8.4 billion Internet of Things (IoT) devices. By 2020, that number is [estimated to surpass 20 billion](https://www.zdnet.com/article/iot-devices-will-outnumber-the-worlds-population-this-year-for-the-first-time/). These types of devices range from consumer devices (e.g., Amazon Echo, smart TVs, smart cameras, door bells) to commercial devices (e.g., building automation systems, keycard entry). All of these devices exhibit behavior on the Internet as they communicate back with their own clouds and user-specified integrations.

#### Types of Network Data

The most detailed type of data that is typically collected on a network is full Packet CAPture (PCAP) data. This information is detailed and contains everything about the communication, including: source address, destination address, protocols used, bytes transferred, and even the raw data (e.g., image, audio file, executable). PCAP data is fine-grained, meaning that there is a record for each frame being transmitted. A typical communication is composed of many individual packets/frames.

If we aggregate PCAP data so that there is one row of data per communication session, we call that flow level data. A simplified example of this relationship is shown in the figure below.

![PCAP_flow_relationship](pcap_vs_flow.png)

For this tutorial, we use data from the University of New South Wales. In a lab environment, they [collected nearly three weeks of IoT data from 21 IoT devices](http://149.171.189.1). They also kept a detailed [list of devices by MAC address](http://149.171.189.1/resources/List_Of_Devices.txt), so we have ground-truth with respect to each IoT device's behavior on the network.

**Our goal is to utilize the behavior exhibited in the network data to classify IoT devices.**

### Data Investigation

Let's first see some of the data. We'll load a PCAP file in using PyShark (a Python wrapper for Tshark).

In [2]:
import pyshark
cap = pyshark.FileCapture("/cwshare/unsw_iot/16-09-27.pcap")

time: 37.3 ms


In [3]:
print(cap[0])

Packet (Length: 156)
Layer ETH:
	Source: 30:8c:fb:2f:e4:b2
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	Type: IPv4 (0x0800)
	Destination: 14:cc:20:51:33:ea
	Address: 14:cc:20:51:33:ea
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	Address: 30:8c:fb:2f:e4:b2
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
Layer IP:
	Source GeoIP: Unknown
	Identification: 0xd29e (53918)
	Protocol: TCP (6)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Header checksum: 0x7fc2 [validation disabled]
	Total Length: 142
	0000 00.. = Differentiated Services Codepoint: Default (0)
	Destination GeoIP Longitude: -77.483803
	Header checksum status: Unverified
	.... 0101 = Header Length: 20 bytes (5)
	.1.. .... = Don't fragment: Set
	Destination GeoIP City: Ashburn, VA
	0... .... = Reserved bit: Not set
	Destination Geo

In [15]:
print(cap[0].ip)

Layer IP:
	Source GeoIP: Unknown
	Total Length: 142
	Source: 192.168.1.106
	Identification: 0xd29e (53918)
	Header checksum: 0x7fc2 [validation disabled]
	Time to live: 64
	Flags: 0x02 (Don't Fragment)
	Destination GeoIP Country: United States
	0000 00.. = Differentiated Services Codepoint: Default (0)
	Destination GeoIP AS Number: AS14618 Amazon.com, Inc.
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Fragment offset: 0
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	Protocol: TCP (6)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	.... 0101 = Header Length: 20 bytes (5)
	Destination: 52.87.241.159
	..0. .... = More fragments: Not set
	Destination GeoIP Latitude: 39.033501
	Destination GeoIP Longitude: -77.483803
	0100 .... = Version: 4
	Header checksum status: Unverified
	Destination GeoIP City: Ashburn, VA
	Destination GeoIP: Ashburn, VA, United States, AS14618 Amazon.com, Inc., 39.033501, -77.483803

time: 1.2 ms


There's really a lot of features there! In addition to having multiple layers (which may differ between packets), there are a number of other issues with working directly with PCAP. Often the payload is encrypted (note the SSL layer in the above example), rendering it useless. The lack of aggregation also makes it difficult to differentiate between packets. What we really care about for this application is what a *session* looks like. In other words, how a Roku interacts with the network is likely quite different than how a Google Home interacts. 

To save time for the tutorial, all three weeks of PCAP data have already been transformed to flow data, and we can load that in to a typical Pandas dataframe. Due to how the data was created, we have a header row (with column names) as well as a footer row. We want to use the header but will skip the footer.

In [4]:
import pandas as pd
pdf = pd.read_csv("/cwshare/unsw_iot/bro/conn.log", sep='\t', skipfooter=1)
print("==> pdf shape: ",pdf.shape)

/opt/conda/envs/pygdf_dev/lib/python3.5/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


==> pdf shape:  (950384, 23)
time: 12.2 s


We can look at what this new aggregated data looks like, and get a better sense of the columns and their data types.

In [3]:
pdf.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_l2_addr,resp_l2_addr
0,1.474553e+09,CIlOTU4kRBDOEJ2zf,192.168.1.241,61725,192.168.1.1,53,udp,dns,-,-,...,-,0,Dc,1,74,0,0,-,70:ee:50:18:34:43,14:cc:20:51:33:ea
1,1.474553e+09,CmV0US1aCCPzrVRz36,192.168.1.193,4425,192.168.1.223,49153,tcp,http,0.008820,196,...,-,0,ShADadfF,5,464,5,461,-,ec:1a:59:83:28:11,ec:1a:59:79:f4:89
2,1.474553e+09,CEXvDL2UPDYnBDtd6h,192.168.1.193,4426,192.168.1.223,49153,tcp,http,0.008664,198,...,-,0,ShADadfF,5,466,5,461,-,ec:1a:59:83:28:11,ec:1a:59:79:f4:89
3,1.474553e+09,CEXDAD42Irgl4M5go8,192.168.1.193,4977,192.168.1.249,49152,tcp,http,0.020995,186,...,-,0,ShADadfF,5,454,5,1438,-,ec:1a:59:83:28:11,00:16:6c:ab:6b:88
4,1.474553e+09,CW1YbA2fZzHrztJ0rl,192.168.1.193,4978,192.168.1.249,49152,tcp,http,0.018730,186,...,-,0,ShADadfF,5,454,5,1438,-,ec:1a:59:83:28:11,00:16:6c:ab:6b:88


time: 30.3 ms


In [4]:
pdf.dtypes

ts                float64
uid                object
id.orig_h          object
id.orig_p           int64
id.resp_h          object
id.resp_p           int64
proto              object
service            object
duration           object
orig_bytes         object
resp_bytes         object
conn_state         object
local_orig         object
local_resp         object
missed_bytes        int64
history            object
orig_pkts           int64
orig_ip_bytes       int64
resp_pkts           int64
resp_ip_bytes       int64
tunnel_parents     object
orig_l2_addr       object
resp_l2_addr       object
dtype: object

time: 2.91 ms


In [5]:
# maybe we rename the columns

time: 407 µs


In [18]:
labels_pdf = pd.read_csv("/cyshare/KDD2018/lab_mac_labels.csv", sep=',')

time: 2.37 ms


In [19]:
labels_pdf.head()

,Device,MAC,Connection,Category
0,Smart Things,d0:52:a8:00:67:5e,Wired,Hub
1,Amazon Echo,44:65:0d:56:cc:d3,Wireless,Speaker
2,Netatmo Welcome,70:ee:50:18:34:43,Wireless,Camera
3,TP-Link Day Night Cloud camera,f4:f2:6d:93:51:f1,Wireless,Camera
4,Samsung SmartCam,00:16:6c:ab:6b:88,Wireless,Camera


time: 8.5 ms


In [20]:
labels_pdf = labels_pdf.assign(CategoryID=(labels_pdf['Category']).astype('category').cat.codes)

time: 2.1 ms


In [21]:
labels_pdf.head()

,Device,MAC,Connection,Category,CategoryID
0,Smart Things,d0:52:a8:00:67:5e,Wired,Hub,6
1,Amazon Echo,44:65:0d:56:cc:d3,Wireless,Speaker,11
2,Netatmo Welcome,70:ee:50:18:34:43,Wireless,Camera,1
3,TP-Link Day Night Cloud camera,f4:f2:6d:93:51:f1,Wireless,Camera,1
4,Samsung SmartCam,00:16:6c:ab:6b:88,Wireless,Camera,1


time: 9.04 ms


In [25]:
merged_pdf = pd.merge(pdf,labels_pdf, how='left', left_on=['orig_l2_addr'], right_on=['MAC'])

time: 621 ms


In [31]:
merged_pdf = merged_pdf.rename(columns = {'Device':'orig_device',
                                          'MAC':'orig_MAC',
                                          'Connection':'orig_connection',
                                          'Category':'orig_category',
                                          'CategoryID':'orig_category_id'})

time: 424 ms


In [32]:
merged_pdf = pd.merge(merged_pdf,labels_pdf, how='left', left_on=['resp_l2_addr'], right_on=['MAC'])

time: 890 ms


In [34]:
merged_pdf.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,orig_device,orig_MAC,orig_connection,orig_category,orig_category_id,resp_device,resp_MAC,resp_connection,resp_category,resp_category_id
0,1.474553e+09,CIlOTU4kRBDOEJ2zf,192.168.1.241,61725,192.168.1.1,53,udp,dns,-,-,...,Netatmo Welcome,70:ee:50:18:34:43,Wireless,Camera,1.0,TPLink Router Bridge LAN (Gateway,14:cc:20:51:33:ea,Wired,Router,10.0
1,1.474553e+09,CmV0US1aCCPzrVRz36,192.168.1.193,4425,192.168.1.223,49153,tcp,http,0.008820,196,...,Belkin wemo motion sensor,ec:1a:59:83:28:11,Wireless,Switch,12.0,Belkin Wemo switch,ec:1a:59:79:f4:89,Wireless,Switch,12.0
2,1.474553e+09,CEXvDL2UPDYnBDtd6h,192.168.1.193,4426,192.168.1.223,49153,tcp,http,0.008664,198,...,Belkin wemo motion sensor,ec:1a:59:83:28:11,Wireless,Switch,12.0,Belkin Wemo switch,ec:1a:59:79:f4:89,Wireless,Switch,12.0
3,1.474553e+09,CEXDAD42Irgl4M5go8,192.168.1.193,4977,192.168.1.249,49152,tcp,http,0.020995,186,...,Belkin wemo motion sensor,ec:1a:59:83:28:11,Wireless,Switch,12.0,Samsung SmartCam,00:16:6c:ab:6b:88,Wireless,Camera,1.0
4,1.474553e+09,CW1YbA2fZzHrztJ0rl,192.168.1.193,4978,192.168.1.249,49152,tcp,http,0.018730,186,...,Belkin wemo motion sensor,ec:1a:59:83:28:11,Wireless,Switch,12.0,Samsung SmartCam,00:16:6c:ab:6b:88,Wireless,Camera,1.0


time: 31.2 ms


In [35]:
merged_pdf.dtypes

ts                  float64
uid                  object
id.orig_h            object
id.orig_p             int64
id.resp_h            object
id.resp_p             int64
proto                object
service              object
duration             object
orig_bytes           object
resp_bytes           object
conn_state           object
local_orig           object
local_resp           object
missed_bytes          int64
history              object
orig_pkts             int64
orig_ip_bytes         int64
resp_pkts             int64
resp_ip_bytes         int64
tunnel_parents       object
orig_l2_addr         object
resp_l2_addr         object
orig_device          object
orig_MAC             object
orig_connection      object
orig_category        object
orig_category_id    float64
resp_device          object
resp_MAC             object
resp_connection      object
resp_category        object
resp_category_id    float64
dtype: object

time: 3.92 ms


In [83]:
orig_comms_pdf = merged_pdf[['ts','id.orig_h','id.orig_p','proto','service','duration','orig_bytes','orig_pkts','orig_ip_bytes',
                             'orig_device','orig_MAC','orig_category','orig_category_id']]
orig_comms_pdf.columns = ['ts','ip','port','proto','service','duration','bytes','pkts','ip_bytes','device','MAC','category','category_id']

time: 103 ms


In [84]:
resp_comms_pdf = merged_pdf[['ts','id.resp_h','id.resp_p','proto','service','duration','resp_bytes','resp_pkts','resp_ip_bytes',
                             'resp_device','resp_MAC','resp_category','resp_category_id']]
resp_comms_pdf.columns = ['ts','ip','port','proto','service','duration','bytes','pkts','ip_bytes','device','MAC','category','category_id']

time: 128 ms


In [209]:
exploded_pdf = pd.concat([orig_comms_pdf,resp_comms_pdf])
print("==> shape = ", exploded_pdf.shape)

==> shape =  (1900768, 13)
time: 366 ms


In [210]:
exploded_pdf.head()

,ts,ip,port,proto,service,duration,bytes,pkts,ip_bytes,device,MAC,category,category_id
0,1.474553e+09,192.168.1.241,61725,udp,dns,-,-,1,74,Netatmo Welcome,70:ee:50:18:34:43,Camera,1.0
1,1.474553e+09,192.168.1.193,4425,tcp,http,0.008820,196,5,464,Belkin wemo motion sensor,ec:1a:59:83:28:11,Switch,12.0
2,1.474553e+09,192.168.1.193,4426,tcp,http,0.008664,198,5,466,Belkin wemo motion sensor,ec:1a:59:83:28:11,Switch,12.0
3,1.474553e+09,192.168.1.193,4977,tcp,http,0.020995,186,5,454,Belkin wemo motion sensor,ec:1a:59:83:28:11,Switch,12.0
4,1.474553e+09,192.168.1.193,4978,tcp,http,0.018730,186,5,454,Belkin wemo motion sensor,ec:1a:59:83:28:11,Switch,12.0


time: 19.9 ms


In [96]:
import numpy as np

time: 614 µs


In [211]:
exploded_pdf['hour_time_bin'] = exploded_pdf['ts'].apply(lambda x: int(np.floor(x/(60*60))))

time: 2.38 s


In [215]:
one_hour_time_bin_pdf = exploded_pdf[['bytes','pkts','ip_bytes','MAC','category_id','hour_time_bin']].groupby(['MAC','category_id','hour_time_bin']).sum()
one_hour_time_bin_pdf = one_hour_time_bin_pdf.reset_index()

time: 302 ms


In [213]:
one_hour_time_bin_pdf.head()

,MAC,category_id,hour_time_bin,pkts,ip_bytes
0,00:16:6c:ab:6b:88,1.0,409598,67929,29754753
1,00:16:6c:ab:6b:88,1.0,409599,568,115454
2,00:16:6c:ab:6b:88,1.0,409600,679,129004
3,00:16:6c:ab:6b:88,1.0,409601,448,106468
4,00:16:6c:ab:6b:88,1.0,409602,549,111263


time: 7.73 ms


In [156]:
pdf_msk = np.random.rand(len(one_hour_time_bin_pdf)) < 0.7

time: 1.41 ms


In [157]:
train_pdf = one_hour_time_bin_pdf[pdf_msk]
test_pdf = one_hour_time_bin_pdf[~pdf_msk]

print("==> train length =",len(train_pdf))
print("==> test length =",len(test_pdf))

==> train length = 5467
==> test length = 2252
time: 4.68 ms


In [159]:
def bytes_packet_vector(row):
    return [row['ip_bytes'],row['pkts'],int(row['category_id'])]

time: 939 µs


In [168]:
train_X = train_pdf[['pkts','ip_bytes']].values
train_Y = train_pdf['category_id'].values

test_X = test_pdf[['pkts','ip_bytes']].values
test_Y = test_pdf['category_id'].values

time: 2.79 ms


In [169]:
import xgboost as xgb

time: 761 µs


In [171]:
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

time: 1.95 ms


In [192]:
param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 13
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = int(len(exploded_pdf.groupby('category_id')['category_id'].nunique()))
param

{'eta': 0.1,
 'max_depth': 13,
 'nthread': 4,
 'num_class': 13,
 'objective': 'multi:softmax',
 'silent': 1}

time: 192 ms


In [205]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 20

time: 953 µs


In [206]:
bst = xgb.train(param, xg_train, num_round, watchlist)

[0]	train-merror:0.102982	test-merror:0.157638
[1]	train-merror:0.092921	test-merror:0.147425
[2]	train-merror:0.087617	test-merror:0.143872
[3]	train-merror:0.083958	test-merror:0.141208
[4]	train-merror:0.080666	test-merror:0.138099
[5]	train-merror:0.07591	test-merror:0.135435
[6]	train-merror:0.073166	test-merror:0.133659
[7]	train-merror:0.069325	test-merror:0.130107
[8]	train-merror:0.067679	test-merror:0.130107
[9]	train-merror:0.06585	test-merror:0.130551
[10]	train-merror:0.064569	test-merror:0.130995
[11]	train-merror:0.062374	test-merror:0.130551
[12]	train-merror:0.059813	test-merror:0.132771
[13]	train-merror:0.057984	test-merror:0.132771
[14]	train-merror:0.056338	test-merror:0.133215
[15]	train-merror:0.054509	test-merror:0.134103
[16]	train-merror:0.054326	test-merror:0.132771
[17]	train-merror:0.052131	test-merror:0.131883
[18]	train-merror:0.050119	test-merror:0.129218
[19]	train-merror:0.047741	test-merror:0.128774
time: 282 ms


In [203]:
pred = bst.predict(xg_test)

time: 1.7 ms


In [204]:
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
error_rate

0.12655417406749556

time: 2.9 ms


In [220]:
exploded_pdf['5min_time_bin'] = exploded_pdf['ts'].apply(lambda x: int(np.floor(x/(60*5))))

five_min_time_bin_pdf = exploded_pdf[['bytes','pkts','ip_bytes','MAC','category_id','5min_time_bin']].groupby(['MAC','category_id','5min_time_bin']).sum()
five_min_time_bin_pdf = five_min_time_bin_pdf.reset_index()

pdf_msk = np.random.rand(len(five_min_time_bin_pdf)) < 0.7

train_pdf = five_min_time_bin_pdf[pdf_msk]
test_pdf = five_min_time_bin_pdf[~pdf_msk]

train_X = train_pdf[['pkts','ip_bytes']].values
train_Y = train_pdf['category_id'].values

test_X = test_pdf[['pkts','ip_bytes']].values
test_Y = test_pdf['category_id'].values

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 13
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = int(len(exploded_pdf.groupby('category_id')['category_id'].nunique()))

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 20

time: 2.88 s


In [221]:
bst = xgb.train(param, xg_train, num_round, watchlist)

[0]	train-merror:0.079665	test-merror:0.091139
[1]	train-merror:0.077493	test-merror:0.089122
[2]	train-merror:0.075222	test-merror:0.087574
[3]	train-merror:0.072412	test-merror:0.087293
[4]	train-merror:0.07034	test-merror:0.085276
[5]	train-merror:0.069005	test-merror:0.084901
[6]	train-merror:0.06749	test-merror:0.084479
[7]	train-merror:0.066175	test-merror:0.084103
[8]	train-merror:0.064581	test-merror:0.083587
[9]	train-merror:0.063844	test-merror:0.082415
[10]	train-merror:0.062529	test-merror:0.082368
[11]	train-merror:0.061293	test-merror:0.081383
[12]	train-merror:0.060875	test-merror:0.081242
[13]	train-merror:0.059898	test-merror:0.08082
[14]	train-merror:0.05942	test-merror:0.080351
[15]	train-merror:0.058444	test-merror:0.080116
[16]	train-merror:0.058005	test-merror:0.079225
[17]	train-merror:0.057647	test-merror:0.079272
[18]	train-merror:0.056949	test-merror:0.078991
[19]	train-merror:0.056112	test-merror:0.078287
time: 2.27 s


In [222]:
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
error_rate

0.07828697406069703

time: 4.25 ms


In [ ]:
exploded_pdf['5min_time_bin'] = exploded_pdf['ts'].apply(lambda x: int(np.floor(x/(60*5))))

five_min_time_bin_pdf = exploded_pdf[['bytes','pkts','ip_bytes','MAC','category_id','5min_time_bin']].groupby(['MAC','category_id','5min_time_bin']).sum()
five_min_time_bin_pdf = five_min_time_bin_pdf.reset_index()

pdf_msk = np.random.rand(len(five_min_time_bin_pdf)) < 0.7

train_pdf = five_min_time_bin_pdf[pdf_msk]
test_pdf = five_min_time_bin_pdf[~pdf_msk]

train_X = train_pdf[['pkts','ip_bytes']].values
train_Y = train_pdf['category_id'].values

test_X = test_pdf[['pkts','ip_bytes']].values
test_Y = test_pdf['category_id'].values

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 13
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = int(len(exploded_pdf.groupby('category_id')['category_id'].nunique()))

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 20